In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sagas.nlu.utils import fix_sents
from sagas.conf.conf import cf
from sagas.nlu.uni_remote import dep_parse
from sagas.nlu.corenlp_parser import get_chunks

def parse_comps(sents, source):
    sents=fix_sents(sents, source)

    engine=cf.engine(source)
    doc_jsonify, resp = dep_parse(sents, source, engine, ['predicts'])
    if len(resp['predicts']) > 0:
        rs=resp['predicts']
    else:
        rs = get_chunks(doc_jsonify)
    return rs

In [45]:
import sagas
import json
from sagas.nlu.uni_remote_viz import list_synsets
def make_map(partcol, textcol):
    return dict(zip(partcol, textcol))
def list_contrast(rs, lang):
    result=[]
    for serial, r in enumerate(rs):
        type_name = r['type']
        df = sagas.to_df(r['domains'], ['rel', 'index', 'text', 'lemma', 'children', 'features'])
        rec={'type':type_name, 'word':r['word'], 'head': r['head'] if 'head' in r else ''}
        rec['domains']=make_map(df['rel'], df['children'])
        rec['synsets']=list_synsets(r, lang, True)
        result.append(rec)
    return result

def procs(sents, source):
    rs=parse_comps(sents, source=source)
    result=list_contrast(rs, source)    
    return result

sents='彼女は 美人な だけでなく 、 頭も いい です 。'
source='ja'
json_r=procs(sents, source)
print(json.dumps(json_r, indent=2, ensure_ascii=False))

.. request is {'lang': 'ja', 'sents': '彼女は美人なだけでなく、頭もいいです。', 'engine': 'knp', 'pipelines': ['predicts']}
words count 4
[
  {
    "type": "predicate",
    "word": "美人なだけでなく、",
    "head": "",
    "domains": {
      "ガ": [
        "彼女"
      ]
    },
    "synsets": [
      {
        "word": "美人なだけでなく、/美人",
        "indicator": "predicate",
        "comments": [
          "smasher.n.02"
        ]
      },
      {
        "word": "彼女",
        "indicator": "ガ",
        "comments": [
          "girlfriend.n.02"
        ]
      }
    ]
  },
  {
    "type": "predicate",
    "word": "いいです。",
    "head": "",
    "domains": {
      "ガ": [
        "頭"
      ]
    },
    "synsets": [
      {
        "word": "いいです。/良い",
        "indicator": "predicate",
        "comments": [
          "well.s.03",
          "beautiful.a.01",
          "stunning.s.04",
          "right.a.05",
          "fine.s.04",
          "good.s.20",
          "good.a.01",
          "bang-up.s.01",
          "good.a.03",
       

In [58]:
from pprint import pprint
syns=[s for c in json_r for s in c['synsets']]
words=[(s['indicator'], s['word']) for s in syns]
pprint(syns)
pprint(words)

[{'comments': ['smasher.n.02'],
  'indicator': 'predicate',
  'word': '美人なだけでなく、/美人'},
 {'comments': ['girlfriend.n.02'], 'indicator': 'ガ', 'word': '彼女'},
 {'comments': ['well.s.03',
               'beautiful.a.01',
               'stunning.s.04',
               'right.a.05',
               'fine.s.04',
               'good.s.20',
               'good.a.01',
               'bang-up.s.01',
               'good.a.03',
               'good.s.16',
               'just.a.01',
               'charitable.s.03',
               'nice.a.01',
               'good.s.06',
               'pleasant.s.02',
               'good.s.12',
               'pleasant.a.01',
               'beautiful.s.02',
               'correct.s.02',
               'decent.s.01',
               'right.a.04',
               'good.s.07',
               'satisfactory.a.01',
               'all_right.s.01',
               'brilliant.s.01',
               'well.a.01'],
  'indicator': 'predicate',
  'word': 'いいです。/良い'},
 {'commen

In [21]:
from sagas.nlu.nlu_cli import retrieve_word_info
rs=retrieve_word_info('get_synsets', "良い", 'ja', '*')
rs

['well.s.03',
 'beautiful.a.01',
 'stunning.s.04',
 'right.a.05',
 'fine.s.04',
 'good.s.20',
 'good.a.01',
 'bang-up.s.01',
 'good.a.03',
 'good.s.16',
 'just.a.01',
 'charitable.s.03',
 'nice.a.01',
 'good.s.06',
 'pleasant.s.02',
 'good.s.12',
 'pleasant.a.01',
 'beautiful.s.02',
 'correct.s.02',
 'decent.s.01',
 'right.a.04',
 'good.s.07',
 'satisfactory.a.01',
 'all_right.s.01',
 'brilliant.s.01',
 'well.a.01']

In [41]:
from sagas.nlu.nlu_cli import get_chains, get_word_sets
rs=get_chains("美人なだけでなく/美人", 'ja', '*')
rs

[{'chain': ['smasher.n.02',
   'woman.n.01',
   'adult.n.01',
   'female.n.02',
   'person.n.01',
   'causal_agent.n.01',
   'organism.n.01',
   'physical_entity.n.01',
   'living_thing.n.01',
   'entity.n.01',
   'whole.n.02',
   'object.n.01'],
  'name': 'smasher.n.02',
  'offset': 10613996}]

In [42]:
get_word_sets("美人なだけでなく/美人", 'ja', '*')

{'data': [{'definition': 'a very attractive or seductive looking woman',
   'domains': {'region': [], 'topic': [], 'usage': []},
   'examples': [],
   'lemmas': {'en': ['smasher',
     'stunner',
     'knockout',
     'beauty',
     'ravisher',
     'sweetheart',
     'peach',
     'lulu',
     'looker',
     'mantrap',
     'dish'],
    'ja': ['シャン',
     'ベイビー',
     'ビューティ',
     'ビューティー',
     '佳人',
     '傾国',
     '傾城',
     '別嬪',
     '契情',
     '小町',
     '小町娘',
     '尤物',
     '弁天',
     '物いう花',
     '物言う花',
     '紅裙',
     '美人',
     '美女',
     '美姫',
     '美形',
     '美玉',
     '蛾眉',
     'もの言う花',
     '辨天',
     '麗人'],
    'zh': []},
   'name': 'smasher.n.02'}],
 'result': 'success'}

In [43]:
get_word_sets("人", 'ja', '*')

{'data': [{'definition': 'a human being',
   'domains': {'region': [], 'topic': [], 'usage': []},
   'examples': ['there was too much for one person to do'],
   'lemmas': {'en': ['person',
     'individual',
     'someone',
     'somebody',
     'mortal',
     'soul'],
    'ja': ['-人', '人', '人間', '個人', '-員', '-者'],
    'zh': ['个人', '个体', '凡人', '某人', '灵魂']},
   'name': 'person.n.01'},
  {'definition': 'any living or extinct member of the family Hominidae characterized by superior intelligence, articulate speech, and erect carriage',
   'domains': {'region': [], 'topic': [], 'usage': []},
   'examples': [],
   'lemmas': {'en': ['homo', 'man', 'human_being', 'human'],
    'ja': ['ヒト', '人', '人間', 'ヒト属'],
    'zh': ['人', '人类']},
   'name': 'homo.n.02'},
  {'definition': 'all of the living human inhabitants of the earth',
   'domains': {'region': [], 'topic': [], 'usage': []},
   'examples': ['all the world loves a lover',
    "she always used `humankind' because `mankind' seemed to slight t

In [4]:
procs('Die Aufnahmen begannen im November.', 'de')

.. request is {'lang': 'de', 'sents': 'Die Aufnahmen begannen im November.', 'engine': 'corenlp', 'pipelines': ['predicts']}
words count 7
[
  {
    "type": "verb_domains",
    "word": "begannen",
    "head": "",
    "domains": {
      "nsubj": [
        "Die",
        "Aufnahmen"
      ],
      "obl": [
        "in",
        "dem",
        "November"
      ],
      "punct": [
        "."
      ]
    },
    "synsets": [
      {
        "word": "begannen/beginnen",
        "indicator": "verb_domains",
        "comments": [
          "get_down.v.07",
          "begin.v.03",
          "begin.v.08"
        ]
      },
      {
        "word": "November",
        "indicator": "obl",
        "comments": [
          "november.n.01"
        ]
      }
    ]
  }
]


In [15]:
from sagas.ofbiz.service_gen import get_service_package, gen_service_stub, get_service_groups
import json

# name='addOrderItemShipGroup'

def print_service(name):
    lines=[]
    schema=gen_service_stub(lines, name)
    print('\n'.join(lines))
    print(json.dumps(schema, indent=2))
print_service('addOrderItemShipGroup')

  /**
   * Creates a new OrderItemShipGroup with maySplit and isGift filled. - OrderItemShipGroup
   *
   * Requires orderId
   * Returns shipGroupSeqId[String], shipGroupSeqId[String]
   */
  Future<OfResult> addOrderItemShipGroup(OrderItemShipGroup ent, ) =>
      client.invoke('addOrderItemShipGroup', ent, {  });
{
  "requires": [
    {
      "name": "orderId",
      "type": "String"
    }
  ],
  "returns": [
    {
      "name": "shipGroupSeqId",
      "type": "String"
    },
    {
      "name": "shipGroupSeqId",
      "type": "String"
    }
  ],
  "parameters": [
    {
      "name": "ent",
      "type": "OrderItemShipGroup"
    }
  ]
}


In [14]:
print_service('getWorkEffortEventsByPeriod')

  /**
   * Get WorkEffort Events by a period specified by periodType attribute (one of the
          java.util.Calendar field values). Return a Map with periodStart as the key and a Collection of events for that period as value
          If filterOutCanceledEvents is set to Boolean(true) then workEfforts with currentStatusId=EVENT_CANCELLED will not be returned.
          To limit the events to a particular partyId, specify the partyId.  To limit the events to a set of partyIds, specify a Collection of partyIds.
         - 
   *
   * Requires start, numPeriods, periodType
   * Returns periods[java.util.List], maxConcurrentEntries[java.lang.Integer]
   */
  Future<OfResult> getWorkEffortEventsByPeriod({String calendarType, String partyId, List<dynamic> partyIds, String facilityId, String fixedAssetId, String workEffortTypeId, @required DateTime start, @required int numPeriods, @required int periodType, bool filterOutCanceledEvents, List<dynamic> entityExprList}) =>
      client.invoke('